<a href="https://colab.research.google.com/github/MassGH2023/Supervised-Machine-Learning-Classification/blob/main/Decision_tree_wine_quality_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/datasets/Wine_Quality_Data.csv", sep=',')

In [3]:
data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [4]:
data.color = data.color.replace({'white': 0, 'red': 1}).astype(int)

<ipython-input-4-770d4169873c>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.color = data.color.replace({'white': 0, 'red': 1}).astype(int)


` pd.get_dummies(df)` also can be used!




In [6]:
features = [x for x in data.columns if x != 'color']

`['fixed_acidity',
 'volatile_acidity',
 'citric_acid',
 'residual_sugar',
 'chlorides',
 'free_sulfur_dioxide',
 'total_sulfur_dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']`

In [9]:
data.shape

(6497, 13)

The `StratifiedShuffleSplit` object in Scikit-learn splits a dataset into train and test subsets, maintaining the same proportion of classes (the same proportion of `y`) as in the original dataset.

In [15]:
strat_shuff_split = StratifiedShuffleSplit(n_splits= 1, test_size= 1000, random_state= 42)

train_idx, test_idx = next(strat_shuff_split.split(data[features], data['color']))

len(train_idx), len(test_idx)

(5497, 1000)

In [17]:
X_train = data.loc[train_idx, features]
y_train = data.loc[train_idx, 'color']

X_test = data.loc[test_idx, features]
y_test = data.loc[test_idx, 'color']

In [21]:
round(y_train.value_counts(normalize= 1),2)

,proportion
color,
0,0.75
1,0.25


In [20]:
round(y_test.value_counts(normalize= 1),2)

,proportion
color,
0,0.75
1,0.25


In [26]:
dtc = DecisionTreeClassifier(random_state= 42)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [39]:
print(f"max depth: {dtc.tree_.max_depth}, number of leaves: {dtc.get_n_leaves()}, count of nodes: {dtc.tree_.node_count}")

max depth: 22, number of leaves: 86, count of nodes: 171


In [43]:
len(dtc.feature_importances_), len(features)

(12, 12)

In [34]:
def measure_error(y_true, y_pred, label):
    return pd.Series({'accuracy':accuracy_score(y_true, y_pred),
                      'precision': precision_score(y_true, y_pred),
                      'recall': recall_score(y_true, y_pred),
                      'f1': f1_score(y_true, y_pred)},
                      name=label)

In [35]:
y_train_pred = dtc.predict(X_train)
y_test_pred = dtc.predict(X_test)

In [37]:
pd.concat([measure_error(y_train, y_train_pred, 'train'),
                              measure_error(y_test, y_test_pred, 'test')],
                              axis=1)

,train,test
accuracy,0.999818,0.984000
precision,0.999261,0.963710
recall,1.000000,0.971545
f1,0.999631,0.967611


**Using grid search with cross validation**


In [44]:
param_grid={'max_depth': range(1, dtc.tree_.max_depth+1,2), 'max_features': range(1, len(features)+1)}
dtgs = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid = param_grid, scoring='accuracy', n_jobs=-1)